In [49]:
from sklearn.datasets import load_files
import numpy as np
import glob
import time
from tensorflow.keras.callbacks import TensorBoard
from IPython.display import display
from keras.models import Sequential, Model
from keras import optimizers
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
import tensorflow as tf
from sklearn.datasets import load_files
from keras.utils import np_utils
from glob import glob
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt
import PIL
from PIL import ImageFile, Image
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from sklearn.metrics import classification_report
import pickle

In [2]:
# Add GPU options
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))
NAME = 'Binary_chest_cnn_64x3_{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
#tensorboard --logdir=logs/ --host localhost --port 8088

In [3]:
dataset_folder_path = r'D:\Documents\ITC\Projects\Data Science\images\Class14_Image11_and_Image1'
train_folder = dataset_folder_path + '/train'
val_folder = dataset_folder_path + '/val'

val_files = glob(test_folder + '/**/*.png')
train_files = glob(train_folder + '/**/*.png')


In [4]:
img_height = img_width = 256
channels = 3

if channels == 3:
    color_mode_ = 'rgb'


(img_height, img_width, channels)

(256, 256, 3)

In [5]:
model = Sequential()

model.add(Conv2D(64, (5,5), input_shape=(img_height, img_width, channels)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
          
model.add(Conv2D(32, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
          
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
          
model.add(Dense(15))
model.add(BatchNormalization())
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 252, 252, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 252, 252, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 252, 252, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 126, 126, 32)      128       
__________

In [6]:
adam = optimizers.Adam(lr = 0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


In [8]:
batch_size = 100
epoch_num = 2


datagen = ImageDataGenerator(rescale=1./255)

# test_datagen = ImageDataGenerator(rescale=1./255)



In [9]:
train_generator = datagen.flow_from_directory(train_folder, color_mode=color_mode_,
                                              target_size=(img_height, img_width),
                                              batch_size=batch_size,
                                              shuffle=True,
                                              class_mode='categorical')

test_generator = datagen.flow_from_directory(test_folder, color_mode=color_mode_,
                                                                        target_size=(img_height, img_width),
                                                                        batch_size=batch_size,
                                                                        class_mode='categorical')


history = model.fit_generator(train_generator, steps_per_epoch=len(train_files)//batch_size, epochs=epoch_num,
                    validation_data=test_generator, validation_steps=len(test_files)//batch_size, verbose = 1)



Found 6476 images belonging to 15 classes.
Found 1620 images belonging to 15 classes.
Epoch 1/2
64/64 [==============================] - 4810s 75s/step - loss: 0.2487 - acc: 0.9332 - val_loss: 0.2491 - val_acc: 0.9333
Epoch 2/2
64/64 [==============================] - 4622s 72s/step - loss: 0.2330 - acc: 0.9335 - val_loss: 0.2196 - val_acc: 0.9333


In [17]:
def load_dataset(path):
    """Returns the path and the Label from the folder"""
    data = load_files(path)
    chest_files = np.array(data['filenames'])
    chest_targets = np_utils.to_categorical(np.array(data['target']), 15)
    return chest_files, chest_targets

# load train, test, and validation datasets
to_test_files, to_test_targets = load_dataset(r'C:\Users\lastm\Documents\ITC\Projects\Data Science\images\image1_for_the_test')


In [26]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(256, 256))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)


In [27]:
to_test_tensors = paths_to_tensor(to_test_files).astype('float32')/255

In [30]:
### Calculate classification accuracy on the test dataset.

# get index of predicted dog breed for each image in test set
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in tqdm(to_test_tensors)]

100%|██████████| 4175/4175 [17:41<00:00,  4.03it/s]


In [33]:
# report test accuracy
to_test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(to_test_targets, axis=1))/len(predictions)
print('Test accuracy: {}%'.format(to_test_accuracy))

Test accuracy: 49.74850299401198%


In [46]:
target_names = [x[78:] for x in glob(to_test_folder + '/**')]
print(classification_report(np.argmax(to_test_targets, axis=1), predictions, target_names=target_names))

C:\Users\lastm\Anaconda3\envs\trump_env\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                    precision    recall  f1-score   support

       Atelectasis       0.00      0.00      0.00       192
      Cardiomegaly       0.00      0.00      0.00        72
     Consolidation       0.00      0.00      0.00        64
             Edema       0.00      0.00      0.00        18
          Effusion       0.06      0.42      0.10       156
         Emphysema       0.00      0.00      0.00        55
          Fibrosis       0.04      0.02      0.03        94
            Hernia       0.00      0.00      0.00        17
      Infiltration       0.00      0.00      0.00       417
              Mass       0.00      0.00      0.00        60
        No Finding       0.70      0.73      0.71      2754
            Nodule       0.00      0.00      0.00       106
Pleural_Thickening       0.00      0.00      0.00        62
         Pneumonia       0.00      0.00      0.00        18
      Pneumothorax       0.07      0.07      0.07        90

         micro avg       0.50      0.5

In [50]:
filename = 'model14_classifier.sav'
pickle.dump(model, open(filename, 'wb'))

Try for predict_generator

In [47]:
dataset_folder_path = r'C:\Users\lastm\Documents\ITC\Projects\Data Science\images'
to_test_folder = dataset_folder_path + '/image1_for_the_test'

to_test_files = glob(to_test_folder + '/**/*.png')

to_test_generator = datagen.flow_from_directory(to_test_folder, color_mode=color_mode_,
                                                                        target_size=(img_height, img_width),
                                                                        batch_size=batch_size,
                                                                        class_mode='categorical')

to_test_generator.reset()

Found 4175 images belonging to 15 classes.


In [13]:
#from sklearn.metrics import classification_report

probabilities = model.predict_generator(to_test_generator, steps = len(to_test_files)//batch_size)

predicted_class_indices=np.argmax(probabilities,axis=1)
# print(classification_report(validation_generator.classes, predicted_class_indices))

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=to_test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

results

KeyboardInterrupt: 

In [ ]:
def load_dataset(path, n_classes):
    """Returns the path and the Label from the folder"""
    data = load_files(path)
    chest_files = np.array(data['filenames'])
    chest_targets = np_utils.to_categorical(np.array(data['target']), n_classes)
    return chest_files, chest_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../imgs/all/train', n_classes)
test_files, test_targets = load_dataset('../imgs/all/test', n_classes)